# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 31 2022 12:03PM,238429,102,PBR-052022-02,"Senseonics, Incorporated",Released
1,May 31 2022 12:00PM,238428,10,8086733,Twinlab Consolidated Corporation,Released
2,May 31 2022 11:57AM,238427,19,ADV80002945,"AdvaGen Pharma, Ltd",Released
3,May 31 2022 11:57AM,238427,19,ADV80002946,"AdvaGen Pharma, Ltd",Released
4,May 31 2022 11:57AM,238427,19,ADV80002947,"AdvaGen Pharma, Ltd",Executing
5,May 31 2022 11:57AM,238427,19,ADV80002948,"AdvaGen Pharma, Ltd",Released
6,May 31 2022 11:57AM,238427,19,ADV80002949,"AdvaGen Pharma, Ltd",Released
7,May 31 2022 11:57AM,238427,19,ADV80002950,"AdvaGen Pharma, Ltd",Released
8,May 31 2022 11:57AM,238427,19,ADV80002951,"AdvaGen Pharma, Ltd",Released
9,May 31 2022 11:57AM,238427,19,ADV80002952,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,238426,Released,1
55,238427,Executing,3
56,238427,Released,7
57,238428,Released,1
58,238429,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238424,NaN,NaN,1.0
238426,NaN,NaN,1.0
238427,NaN,3.0,7.0
238428,NaN,NaN,1.0
238429,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238316,0.0,9.0,0.0
238317,0.0,9.0,1.0
238319,0.0,0.0,1.0
238334,0.0,7.0,7.0
238341,0.0,0.0,35.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238316,0,9,0
238317,0,9,1
238319,0,0,1
238334,0,7,7
238341,0,0,35


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238316,0,9,0
1,238317,0,9,1
2,238319,0,0,1
3,238334,0,7,7
4,238341,0,0,35


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238316,,9,
1,238317,,9,1
2,238319,,,1
3,238334,,7,7
4,238341,,,35


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated"
1,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation
2,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd"
12,May 31 2022 11:49AM,238426,18,Innova Softgel LLC
13,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated"
14,May 31 2022 11:31AM,238423,10,"Methapharm, Inc."
23,May 31 2022 11:31AM,238422,10,"Methapharm, Inc."
46,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated"
47,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC
48,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",,,1
1,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,,,1
2,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",,3,7
3,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,,,1
4,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",,,1
5,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",,,9
6,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",,,23
7,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",,,1
8,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,,1,
9,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",1,,
1,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,1,,
2,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",7,3,
3,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,1,,
4,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",1,,
5,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",9,,
6,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",23,,
7,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",1,,
8,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,,1,
9,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",1,,
1,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,1,,
2,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",7,3,
3,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,1,,
4,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",1.0,NaN,NaN
1,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,1.0,NaN,NaN
2,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",7.0,3.0,NaN
3,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,1.0,NaN,NaN
4,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",1.0,0.0,0.0
1,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,1.0,0.0,0.0
2,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",7.0,3.0,0.0
3,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,1.0,0.0,0.0
4,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6198271,520.0,78.0,12.0
102,715216,2.0,1.0,0.0
12,953397,77.0,0.0,0.0
15,714952,2.0,18.0,0.0
18,1191971,5.0,0.0,0.0
19,715125,13.0,5.0,4.0
20,1191942,52.0,8.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6198271,520.0,78.0,12.0
1,102,715216,2.0,1.0,0.0
2,12,953397,77.0,0.0,0.0
3,15,714952,2.0,18.0,0.0
4,18,1191971,5.0,0.0,0.0
5,19,715125,13.0,5.0,4.0
6,20,1191942,52.0,8.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,520.0,78.0,12.0
1,102,2.0,1.0,0.0
2,12,77.0,0.0,0.0
3,15,2.0,18.0,0.0
4,18,5.0,0.0,0.0
5,19,13.0,5.0,4.0
6,20,52.0,8.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,520.0
1,102,Released,2.0
2,12,Released,77.0
3,15,Released,2.0
4,18,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,18,19,20
Status,,,,,,,
Completed,12.0,0.0,0.0,0.0,0.0,4.0,0.0
Executing,78.0,1.0,0.0,18.0,0.0,5.0,8.0
Released,520.0,2.0,77.0,2.0,5.0,13.0,52.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,18,19,20
0,Completed,12.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,78.0,1.0,0.0,18.0,0.0,5.0,8.0
2,Released,520.0,2.0,77.0,2.0,5.0,13.0,52.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,18,19,20
0,Completed,12.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,78.0,1.0,0.0,18.0,0.0,5.0,8.0
2,Released,520.0,2.0,77.0,2.0,5.0,13.0,52.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()